<a href="https://colab.research.google.com/github/jxb5778/AmazonCustomerReviews/blob/master/LDA_Topic_Model_20_News_Group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

import nltk
from nltk import word_tokenize       
from nltk.corpus import stopwords   
from nltk.stem import WordNetLemmatizer 

import re


In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.6MB/s 
     |████████████████████████████████| 552kB 15.0MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=1e30b87692e9c7522cb859765102ff85ee782d58d890d813bbe8381f2cf40182
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=ceaf068f2c8a9196043c03e5e47a14ef6f60fd6e3e9dfc1b99177f05da322031
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [3]:
import pyLDAvis
import pyLDAvis.gensim

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from sklearn.datasets import fetch_20newsgroups


data, _ = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'),
                             return_X_y=True)

In [6]:
print(len(data))

11314


In [7]:
print(data[5])

 
 
I read somewhere, I think in Morton Smith's _Jesus the Magician_, that
old Lazarus wasn't dead, but going in the tomb was part of an initiation
rite for a magi-cult, of which Jesus was also a part.   It appears that
a 3-day stay was normal.   I wonder .... ?


In [8]:
class LemmaTokenizer(object):
    def __init__(self, token_pattern=r"(?u)\b\w\w+\b", stop='english'):
        self.wnl = WordNetLemmatizer()
        self.token_pattern = re.compile(token_pattern)
        self.stop = set(stopwords.words(stop))
        
    def __call__(self, doc):
        return [
                self.wnl.lemmatize(t) for t in word_tokenize(doc) 
                if len(t) > 2 
                and re.match("[a-z].*",t) 
                and re.match(self.token_pattern, t) 
                and t not in self.stop
                ]

In [9]:
tokenizer = LemmaTokenizer()

In [11]:
data_tokenized = [tokenizer(doc) for doc in data]

In [12]:
bigram = gensim.models.Phrases(data_tokenized, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [13]:
texts = [bigram_mod[doc] for doc in data_tokenized]

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(texts)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 3), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)]]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=20, 
                                      random_state=100,
                                      chunksize=100,
                                      passes=10,
                                      alpha='asymmetric',
                                      eta='auto',
                                      per_word_topics=True)

In [16]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.028*"key" + 0.016*"use" + 0.012*"system" + 0.010*"public" + 0.008*"information" + 0.008*"chip" + 0.007*"stream" + 0.007*"security" + 0.007*"used" + 0.006*"user"'), (1, '0.033*"drive" + 0.030*"system" + 0.021*"disk" + 0.016*"card" + 0.013*"use" + 0.011*"monitor" + 0.009*"mode" + 0.009*"data" + 0.008*"memory" + 0.008*"standard"'), (2, '0.018*"said" + 0.014*"child" + 0.012*"people" + 0.012*"one" + 0.010*"say" + 0.010*"know" + 0.009*"time" + 0.009*"could" + 0.009*"woman" + 0.008*"day"'), (3, '0.030*"perfect" + 0.009*"immune_system" + 0.009*"club" + 0.009*"food" + 0.009*"percentage" + 0.008*"soon" + 0.007*"hockey" + 0.007*"music" + 0.007*"illness" + 0.007*"eye"'), (4, '0.016*"would" + 0.013*"people" + 0.012*"one" + 0.011*"think" + 0.011*"say" + 0.009*"know" + 0.009*"see" + 0.009*"thing" + 0.008*"believe" + 0.008*"like"'), (5, '0.012*"war" + 0.010*"stand" + 0.010*"church" + 0.010*"crime" + 0.009*"speaking" + 0.009*"attack" + 0.009*"peace" + 0.008*"murder" + 0.007*"criminal" + 0.006*"

In [ ]:
log_perplexity = lda_model.log_perplexity(corpus)

coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()


In [19]:
print('\nPerplexity: ', log_perplexity)  # Measure of how well the model fits the data. A lower score is better.
print('\nCoherence Score: ', coherence_lda) # Measure for how interpretable are the topics


Perplexity:  -9.451096860100199

Coherence Score:  -4.461357487597607


In [20]:
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.227295 -0.018757       1        1  20.247702
7     -0.187703 -0.120820       2        1   8.780373
8     -0.184851  0.105776       3        1   8.529989
2     -0.158237 -0.166179       4        1   7.256286
17    -0.029372  0.017189       5        1   6.977035
19    -0.010147  0.284770       6        1   6.108170
0     -0.012392  0.165257       7        1   5.786480
15    -0.177906 -0.003530       8        1   5.772578
13    -0.120985 -0.100433       9        1   5.310601
1     -0.029709  0.127469      10        1   4.645679
11     0.068438  0.125390      11        1   3.411252
5      0.072355 -0.073545      12        1   3.252424
3      0.078058 -0.089511      13        1   2.470665
14     0.097517  0.002013      14        1   2.096504
10     0.120992 -0.043848      15        1   1.822792
6      0.110204 -0.068539      16        1   1.656856
18     0.136695 -0.064912      17        1   1.642435
12     0.141199 -0.013249      18        1   1.533631
9      0.156674 -0.011821      19        1   1.368608
16     0.156465 -0.052720      20        1   1.329943, topic_info=          Term         Freq        Total Category  logprob  loglift
242       file  2322.000000  2322.000000  Default  30.0000  30.0000
131      drive  1212.000000  1212.000000  Default  29.0000  29.0000
142        key  1287.000000  1287.000000  Default  28.0000  28.0000
2885      game  1444.000000  1444.000000  Default  27.0000  27.0000
277    program  2111.000000  2111.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
36997    goose    45.998497    46.351467  Topic20  -5.4189   4.3124
2795   payload    48.875683    49.259499  Topic20  -5.3583   4.3122
2518       lab    48.943687    49.329647  Topic20  -5.3569   4.3122
4338    placed    75.274231    86.930748  Topic20  -4.9264   4.1761
6977    spring    74.335365    88.859085  Topic20  -4.9390   4.1416

[1058 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
5641     12  0.999330     absolute
5382     13  0.990760  accelerator
414       3  0.368484       access
414       7  0.629055       access
2422      9  0.998924     accident
...     ...       ...          ...
181      19  0.002114         year
181      20  0.001057         year
5720      9  0.990605       yellow
7729     16  1.000165         zero
2384     12  1.000535         zone

[3068 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 9, 3, 18, 20, 1, 16, 14, 2, 12, 6, 4, 15, 11, 7, 19, 13, 10, 17])

In [ ]:

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3, coherence='c_v'):
    """
    Compute log_perplexity and coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    id2word: Gensim id2word
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    log_perplexity_values: Log perplexity values corresponding to the LDA model with respective number of topics
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    log_perplexity_values = list()
    coherence_values = list()
    model_list = list()

    for num_topics in range(start, limit, step):
        model = gensim.models.LdaMulticore(corpus=corpus,
                                      id2word=dictionary,
                                      num_topics=20, 
                                      random_state=100,
                                      chunksize=100,
                                      passes=10,
                                      alpha='asymmetric',
                                      eta='auto',
                                      per_word_topics=True)
        model_list.append(model)

        log_perplexity = model.log_perplexity(corpus)
        log_perplexity_values.append(log_perplexity)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, log_perplexity_values, coherence_values


In [ ]:
# Will take a long time
model_list, log_perplexity_values, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=2, limit=40, step=6)

In [ ]:
import pandas as pd


df = pd.DataFrame({
    'num_topics': range(2, 40, 6),
    'log_perplexity': log_perplexity_values,
    'coherence': coherence_values
    
})


In [ ]:
import plotly.express as px


fig = px.scatter(df, x='coherence', y='log_perplexity', hover_data=['num_topics'])
fig.show()
